# Load Models

In [2]:
!pip install peft --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 7.2 MB/s eta 0:00:00


In [9]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM,AutoTokenizer
from transformers import pipeline

base_model = "Sakonii/distilgpt2-nepali"
adapter_model = "Bibek1129/distilgpt2-nepali-multiple-qs-generator"
# adapter_model = "Bibek1129/distilgpt2-nepali-single-qs-generator"

tokenizer = AutoTokenizer.from_pretrained(base_model)

config = PeftConfig.from_pretrained(adapter_model)
model = AutoModelForCausalLM.from_pretrained(base_model)
model = PeftModel.from_pretrained(model, adapter_model)
model = model.merge_and_unload()

tokenizer_config.json:   0%|          | 0.00/413 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.13M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/layer.py:861: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/layer.py:852: UserWarning: fan_in_fan_out is set to True but the target module is `torch.nn.Linear`. Setting fan_in_fan_out to False.
  warnings.warn(


# Prompt

In [42]:
prompt = """तपाईं प्रश्नहरू उत्पन्न गर्ने मोडेल हुनुहुन्छ। तपाइँलाई एक सन्दर्भ दिइएको हुन्छ र तपाइँ त्यसमा आधारित प्रश्नहरू उत्पन्न गर्नुहुन्छ।

### सन्दर्भ:
राजनीति 'शहरका मामिलाहरू') गतिविधिहरूको सेट हो जुन समूहहरूमा निर्णय गर्न वा व्यक्तिहरू बीचको शक्ति सम्बन्धका अन्य रूपहरू, जस्तै स्रोत वा स्थितिको वितरणसँग सम्बन्धित छ। राजनीति र सरकारको अध्ययन गर्ने सामाजिक विज्ञानको शाखालाई राजनीति विज्ञान भनिन्छ।
यसलाई "राजनीतिक समाधान" को सन्दर्भमा सकारात्मक रूपमा प्रयोग गर्न सकिन्छ जुन सम्झौता र अहिंसात्मक छ, वा वर्णनात्मक रूपमा "सरकारको कला वा विज्ञान" को रूपमा, तर प्राय: नकारात्मक अर्थ पनि बोक्छ। अवधारणालाई विभिन्न तरिकामा परिभाषित गरिएको छ, र यसलाई
व्यापक रूपमा प्रयोग गर्ने वा सीमित रूपमा, प्रायोगिक वा सामान्य रूपमा, र यसको लागि द्वन्द्व वा सहयोग बढी आवश्यक छ कि छैन भन्ने बारेमा विभिन्न दृष्टिकोणहरूमा मौलिक रूपमा फरक फरक विचारहरू छन्।

### प्रश्नहरू:
"""

# Inference

In [43]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=64)

In [44]:
def format_output(prompt,pipe):
  inference = pipe(prompt)[0]["generated_text"]

  # Select after प्रश्नहरू: and break line after each ?
  inference = inference.split("प्रश्नहरू:")[-1].replace("?","?\n")

  # Remove last incomplete question
  index = inference.rfind("?")
  inference = inference[:index+1]
  return inference


In [48]:
print(format_output(prompt, pipe))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



 राजनीतिशास्त्रले मानिसहरूलाई केको रूपमा देख्छ?
 राजनीतिशास्त्र प्राय: कुन प्रकारको अभ्याससँग सम्बन्धित छ?
 राजनीतिशास्त्रले मानिसलाई केको रूपमा देख्छ?
 राजनीति विज्ञानमा केको भूमिका निर्भर छ?
 राजनीतिक अर्थशास्त्रको शाखालाई कसरी प्रभावित गरेर समाजलाई सांस्कृतिक परिभाषामा के असर हुन्छ,?
